## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nanortalik,GL,60.1432,-45.2371,31.10,93,2,12.57,clear sky
1,1,Hilo,US,19.7297,-155.0900,73.40,64,75,9.08,light rain
2,2,Cidreira,BR,-30.1811,-50.2056,72.90,91,100,15.17,overcast clouds
3,3,Port Augusta,AU,-32.5000,137.7667,82.99,65,0,21.85,clear sky
4,4,Rikitea,PF,-23.1203,-134.9692,80.01,69,87,4.25,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hilo,US,19.7297,-155.0900,73.40,64,75,9.08,light rain
2,2,Cidreira,BR,-30.1811,-50.2056,72.90,91,100,15.17,overcast clouds
3,3,Port Augusta,AU,-32.5000,137.7667,82.99,65,0,21.85,clear sky
4,4,Rikitea,PF,-23.1203,-134.9692,80.01,69,87,4.25,light rain
7,7,Zaraza,VE,9.3503,-65.3245,70.68,92,52,4.23,broken clouds
8,8,Poum,NC,-20.2333,164.0167,80.11,79,98,11.27,overcast clouds
9,9,Chuy,UY,-33.6971,-53.4616,66.33,83,73,8.05,broken clouds
10,10,Puerto Ayora,EC,-0.7393,-90.3518,77.00,90,67,1.01,broken clouds
11,11,Gold Coast,AU,-28.0000,153.4333,82.99,61,40,12.66,scattered clouds
12,12,Port Alfred,ZA,-33.5906,26.8910,71.01,82,0,5.01,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hilo,US,73.40,light rain,19.7297,-155.0900,
2,Cidreira,BR,72.90,overcast clouds,-30.1811,-50.2056,
3,Port Augusta,AU,82.99,clear sky,-32.5000,137.7667,
4,Rikitea,PF,80.01,light rain,-23.1203,-134.9692,
7,Zaraza,VE,70.68,broken clouds,9.3503,-65.3245,
8,Poum,NC,80.11,overcast clouds,-20.2333,164.0167,
9,Chuy,UY,66.33,broken clouds,-33.6971,-53.4616,
10,Puerto Ayora,EC,77.00,broken clouds,-0.7393,-90.3518,
11,Gold Coast,AU,82.99,scattered clouds,-28.0000,153.4333,
12,Port Alfred,ZA,71.01,clear sky,-33.5906,26.8910,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    response=requests.get(base_url,params=params)
    #print(response.url)
    #commented out the print statement above when adding to github so API key isn't exposed
    hotels=response.json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hilo,US,73.40,light rain,19.7297,-155.0900,
2,Cidreira,BR,72.90,overcast clouds,-30.1811,-50.2056,
3,Port Augusta,AU,82.99,clear sky,-32.5000,137.7667,
4,Rikitea,PF,80.01,light rain,-23.1203,-134.9692,
7,Zaraza,VE,70.68,broken clouds,9.3503,-65.3245,
...,...,...,...,...,...,...,...
700,Gondanglegi,ID,84.02,overcast clouds,-7.7479,112.0132,
702,Alofi,NU,78.80,overcast clouds,-19.0595,-169.9187,
703,Tambacounda,SN,68.00,clear sky,13.7539,-13.7586,
704,Santiago De Cao,PE,70.21,broken clouds,-7.9589,-79.2392,


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map.
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))